<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Data_collection_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

!pip install wikidata --quiet

import matplotlib.pyplot as plt
import pandas as pd
from wikidata.client import Client
import requests

from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned (2).tsv


In [5]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

In [35]:
def get_sitelinks(wikidata_url):

  # Extract the id of the item
  entity_id = extract_entity_id(wikidata_url)

  # Extract information of the item
  client = Client()
  item = client.get(entity_id, load=True)

  # Extract the sitelinks of the item
  sitelinks = item.data.get("sitelinks", {})

  return sitelinks



def get_first_1000_characters(wikidata_url):

    # Initialisation of parameters, list and dictionnary

    selected_languages = ["en", "fr", "es", "zh", "ar", "hi", "ru", "pt", "ja", "sw"]
    dict_lang = {}

    # Extraction of the Wikipedia pages informations, for the selected item
    sitelinks = get_sitelinks(wikidata_url)

    # Extraction of the wikipedia page, for every languages
    for lang in selected_languages:
        lang_wiki = sitelinks.get(f"{lang}wiki")
        if not lang_wiki:
            continue

        title = lang_wiki["title"]

        api_url = f"https://{lang}.wikipedia.org/w/api.php"
        params = {
            "action": "query",
            "prop": "extracts",
            "explaintext": True,
            "titles": title,
            "format": "json",
            "redirects": 1
        }

        try:
            res = requests.get(api_url, params=params).json()
            page = next(iter(res["query"]["pages"].values()))
            text = page.get("extract", "")
            dict_lang[lang] = text[:1000] # Selection of the first 1000 characters only

        except Exception as e:
            continue

    return dict_lang

# Test
wikidata_url = train_df.iloc[0,0]
dict_languages = get_first_1000_characters(wikidata_url)

In [36]:
!pip install googletrans==4.0.0-rc1 --quiet

from googletrans import Translator

def translate_to_english(sentence, lang):

    translator = Translator()
    try:
        # Translation
        translation = translator.translate(sentence, src=lang, dest='en')
        return translation.text
    except Exception as e:
        return None

# Test
#translate_to_english('essai', 'fr')

In [39]:
dict_english = {}

for lang, text in dict_languages.items():
  dict_english[lang] = translate_to_english(text, lang)

# Test
dict_english

{'en': "Miguel Juan Sebastián Piñera Echenique (Spanish: [miˈɣel ˈxwan seβasˈtjam piˈɲeɾa etʃeˈnike] ; 1 December 1949 – 6 February 2024) was a Chilean businessman and politician who served as President of Chile from 2010 to 2014 and again from 2018 to 2022. The son of a Christian Democratic politician and diplomat, he studied business administration at the Pontifical Catholic University of Chile and economics at Harvard University. At the time of his death, he had an estimated net worth of US$2.7 billion, according to Forbes, making him the third richest person in Chile and the 1177th richest person in the world.\nA member of the liberal-conservative National Renewal party, he served as a senator for the East Santiago district from 1990 to 1998, running for the presidency in the 2005 election, which he lost to Michelle Bachelet, and again, successfully, in 2010. As a result, he became Chile's first conservative president to be democratically elected since 1958, and the first to hold t

In [42]:
# Comparison between english texts, and translated texts :

# Jaccard Similitary

def jaccard_similarity(text1, text2):
    set1 = set(text1.lower().split())
    set2 = set(text2.lower().split())

    intersection = len(set1 & set2)
    union = len(set1 | set2)

    return intersection / union

# Levenstein Distance

!pip install Levenshtein --quiet
import Levenshtein

def levenshtein_distance(text1, text2):
    return Levenshtein.distance(text1, text2)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.0 MB/s eta 0:00:00
Similarité Jaccard : 0.16666666666666666
Distance de Levenshtein : 17


In [45]:
for lang, text in dict_english.items():
  print(f"Levensetien Distance : en - {lang} : ", levenshtein_distance(dict_languages['en'], dict_languages[f'{lang}']))

Levensetien Distance : en - en :  0
Levensetien Distance : en - fr :  780
Levensetien Distance : en - es :  789
Levensetien Distance : en - zh :  931
Levensetien Distance : en - ar :  925
Levensetien Distance : en - hi :  948
Levensetien Distance : en - ru :  884
Levensetien Distance : en - pt :  774
Levensetien Distance : en - ja :  982
Levensetien Distance : en - sw :  841
